In [ ]:
!pip install scispacy https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bionlp13cg_md-0.5.1.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 MB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of en-ner-bionlp13cg-md to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 10.4 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl.metadata (8.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.3 MB/s eta 0:00:00
Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 

In [ ]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.7 MB/s eta 0:00:00


In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 34.8 MB/s eta 0:00:00


In [ ]:
!pip install python-docx groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.5 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
import os
import requests
import spacy
from io import BytesIO
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import pagesizes, colors
import fitz
from groq import Groq
from docx import Document
import PyPDF2
from google.colab import files

In [ ]:
nlp = spacy.load("en_ner_bionlp13cg_md")

In [ ]:
os.environ["GROQ_API_KEY"] = ""
api_key = os.getenv("GROQ_API_KEY")
client = Groq(api_key=api_key)

In [40]:
def get_gene_info_ensembl(gene_name):
    """
    Fetches gene information from the Ensembl REST API.
    """
    url = f"https://rest.ensembl.org/lookup/symbol/homo_sapiens/{gene_name}?content-type=application/json"

    response = requests.get(url)

    if response.status_code == 200:
        gene_data = response.json()

        gene_info = {
            "Gene Name": gene_data.get("display_name", "N/A"),
            "Gene Symbol": gene_data.get("display_name", "N/A"),
            "Gene ID": gene_data.get("id", "N/A"),
            "Chromosome": gene_data.get("seq_region_name", "N/A"),
            "Start": gene_data.get("start", "N/A"),
            "End": gene_data.get("end", "N/A")
        }
        return gene_info
    else:
        print(f"Error fetching data from Ensembl for gene: {gene_name}")
        return None

In [41]:
def get_gene_function(gene_name):
    """
    Fetches the gene function from mygene.info API.
    """
    url = f"https://mygene.info/v3/query?q={gene_name}&fields=symbol,name,summary"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'hits' in data and len(data['hits']) > 0:
            gene_info = data['hits'][0]
            return {
                "symbol": gene_info.get("symbol", "N/A"),
                "name": gene_info.get("name", "N/A"),
                "summary": gene_info.get("summary", "No function available")
            }
    return None

In [72]:
def get_filtered_mutation_data_ensembl(gene_name, mutation_limit=5, mutation_type_filters=["stop_gained"]):
    """
    Fetches mutation data for a gene from Ensembl and applies filters for each mutation type separately.
    Each mutation type gets its own limit.
    """
    gene_info = get_gene_info_ensembl(gene_name)

    if gene_info:
        gene_id = gene_info["Gene ID"]
        print(f"Fetching mutations for Gene ID: {gene_id}")

        url = f"https://rest.ensembl.org/overlap/id/{gene_id}?feature=variation;content-type=application/json"

        response = requests.get(url)

        if response.status_code == 200:
            mutation_data = response.json()
            print(f"Received {len(mutation_data)} mutations from Ensembl.")

            mutations = []
            seen_variants = set()
            count_per_mutation_type = {mt: 0 for mt in mutation_type_filters}

            for mutation in mutation_data:
                variation_id = mutation.get("id", "N/A")
                if variation_id in seen_variants:
                    continue

                consequence_type = mutation.get("consequence_type", [])

                if isinstance(consequence_type, str):
                    consequence_type = [consequence_type]

                for mt in mutation_type_filters:
                    if any(mt.lower() in consequence.lower() for consequence in consequence_type):
                        if count_per_mutation_type[mt] >= mutation_limit:
                            continue

                        seq_region_name = mutation.get("seq_region_name", "N/A")
                        allele_string = mutation.get("allele_string", "N/A")

                        if allele_string == "N/A":
                            allele_string = mutation.get("alleles", "N/A")

                        if isinstance(allele_string, list):
                            allele_string = ', '.join(allele_string)

                        mutation_dict = {
                            "Variation": variation_id,
                            "Location": seq_region_name,
                            "Allele": allele_string,
                            "Consequence": '/'.join(consequence_type),
                        }
                        mutations.append(mutation_dict)

                        seen_variants.add(variation_id)

                        count_per_mutation_type[mt] += 1

                if all(count >= mutation_limit for count in count_per_mutation_type.values()):
                    break

            for mt in mutation_type_filters:
                print(f"Found {count_per_mutation_type[mt]} mutations for {mt}.")

            print(f"Total filtered mutations: {len(mutations)}")

            return mutations if mutations else "No mutations found that match the criteria."
        else:
            print(f"Error fetching mutation data from Ensembl for gene ID: {gene_id}, status code: {response.status_code}")
            return "Error fetching mutation data from Ensembl."
    else:
        print(f"Gene information not found for {gene_name}")
        return "Gene information not found."

In [73]:
def extract_text_from_pdf(pdf_content):
    doc = fitz.open(stream=pdf_content, filetype="pdf")
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [74]:
def extract_entities(text):
    doc = nlp(text)
    entities = {
        "genes": [],
    }

    for ent in doc.ents:
        if ent.label_ == "GENE_OR_GENE_PRODUCT":
            entities["genes"].append(ent.text)

    return entities

In [75]:
def chatbot_with_groq(question, context):
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are a helpful genetic counseling assistant."},
            {"role": "user", "content": question},
            {"role": "system", "content": f"Context: {context}"}
        ],
        model="llama3-8b-8192",
    )
    return chat_completion.choices[0].message.content

In [76]:
def extract_genes_from_txt(file_path):
    """
    Extract genes from a plain text (.txt) file.
    """
    with open(file_path, "r") as file:
        text = file.read()
    entities = extract_entities(text)
    return entities["genes"]

In [77]:
def extract_genes_from_pdf(file_path):
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        entities = extract_entities(text)
        return entities["genes"]

In [78]:
def extract_genes_from_docx(file_path):
    doc = Document(file_path)
    text = ""
    for para in doc.paragraphs:
        text += para.text
    entities = extract_entities(text)
    return entities["genes"]

In [79]:
def extract_genes_from_document(file_path):
    """
    Extract genes from a document based on its file extension.
    Supports PDF, DOCX, and TXT files.
    """
    if file_path.endswith(".pdf"):
        return extract_genes_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        return extract_genes_from_docx(file_path)
    elif file_path.endswith(".txt"):
        return extract_genes_from_txt(file_path)
    else:
        print("Unsupported file type. Please upload a PDF, DOCX, or TXT file.")
        return []

In [80]:
def get_file_path_colab():
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]
    return file_path

In [81]:
def wrap_text(text, width, font_size, font_name="Helvetica", x_offset=100):
    """
    Wrap the text to fit within the given width and adjust for right padding.
    """
    c = canvas.Canvas(BytesIO(), pagesize=letter)
    c.setFont(font_name, font_size)
    words = text.split(" ")
    lines = []
    current_line = ""

    for word in words:
        test_line = current_line + " " + word if current_line else word
        text_width = c.stringWidth(test_line, font_name, font_size)
        if text_width <= width - x_offset - 100:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word

    if current_line:
        lines.append(current_line)

    return lines

def draw_underline(c, text, x_position, y_position, font_name="Helvetica-Bold", font_size=12, x_offset=100, line_padding=0):
    """
    Draw an underline beneath the text at the given position with more padding from the right.
    """
    text_width = c.stringWidth(text, font_name, font_size)
    c.drawString(x_position, y_position, text)
    c.line(x_position, y_position - 2, x_position + text_width + line_padding, y_position - 2)

def draw_full_line(c, y_position, width=500, x_offset=100, line_padding=0):
    """
    Draw a full-width horizontal line to separate gene sections with more right padding.
    """
    c.setStrokeColorRGB(0, 0, 0)
    c.setLineWidth(1)
    c.line(x_offset, y_position, x_offset + width + line_padding, y_position)

def generate_report(genes_data):
    """
    Generate a combined PDF report for multiple genes with improved formatting and page handling.
    """
    buffer = BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    width, height = letter
    c.setFont("Helvetica", 14)

    header_height = 50
    footer_height = 40
    content_top_margin = 80
    content_bottom_margin = footer_height + 20
    x_offset = 120

    y_position = height - header_height - 20

    c.setFillColorRGB(0.2, 0.4, 0.6)
    c.drawString(x_offset, y_position, "Genetic Counseling Report")
    y_position -= 30

    def check_page_break(y_position):
        """Check if the current y_position is too low and a page break is needed"""
        if y_position < content_bottom_margin:
            c.showPage()
            c.setFont("Helvetica", 14)
            c.setFillColorRGB(0.2, 0.4, 0.6)
            y_position = height - header_height - 20
            c.drawString(x_offset, y_position, "Genetic Counseling Report")
            y_position -= 30

            c.setFont("Helvetica", 10)
        return y_position

    for gene_data in genes_data:
        gene_info, gene_function, mutations = gene_data

        c.setFont("Helvetica-Bold", 12)
        draw_underline(c, f"Gene Information: {gene_info.get('Gene Symbol', 'N/A')}", x_offset, y_position, x_offset=x_offset)
        y_position -= 30

        y_position = check_page_break(y_position)

        c.setFont("Helvetica", 10)
        if gene_info:
            for key, value in gene_info.items():
                lines = wrap_text(f"{key}: {value}", width, 10, x_offset=x_offset)
                for line in lines:
                    y_position = check_page_break(y_position)
                    c.drawString(x_offset, y_position, line)
                    y_position -= 15
        else:
            c.drawString(x_offset, y_position, "No gene information available.")
            y_position -= 20

        y_position -= 20

        c.setFont("Helvetica-Bold", 12)
        draw_underline(c, "Gene Function:", x_offset, y_position, x_offset=x_offset)
        y_position -= 30
        y_position = check_page_break(y_position)
        c.setFont("Helvetica", 10)

        if gene_function:
            lines = wrap_text(f"Name: {gene_function['name']}", width, 10, x_offset=x_offset)
            for line in lines:
                y_position = check_page_break(y_position)
                c.drawString(x_offset, y_position, line)
                y_position -= 15

            lines = wrap_text(f"Symbol: {gene_function['symbol']}", width, 10, x_offset=x_offset)
            for line in lines:
                y_position = check_page_break(y_position)
                c.drawString(x_offset, y_position, line)
                y_position -= 15

            lines = wrap_text(f"Function Summary: {gene_function['summary']}", width, 10, x_offset=x_offset)
            for line in lines:
                y_position = check_page_break(y_position)
                c.drawString(x_offset, y_position, line)
                y_position -= 15
        else:
            c.drawString(x_offset, y_position, "No gene function information available.")
            y_position -= 20

        y_position -= 40

        c.setFont("Helvetica-Bold", 12)
        draw_underline(c, "Mutation Interpretation:", x_offset, y_position, x_offset=x_offset)
        y_position -= 30
        y_position = check_page_break(y_position)
        c.setFont("Helvetica", 10)

        if mutations:
            for mutation in mutations:
                lines = wrap_text(f"Variation: {mutation['Variation']}", width, 10, x_offset=x_offset)
                for line in lines:
                    y_position = check_page_break(y_position)
                    c.drawString(x_offset, y_position, line)
                    y_position -= 15

                lines = wrap_text(f"Location: {mutation['Location']}", width, 10, x_offset=x_offset)
                for line in lines:
                    y_position = check_page_break(y_position)
                    c.drawString(x_offset, y_position, line)
                    y_position -= 15

                lines = wrap_text(f"Consequence: {mutation['Consequence']}", width, 10, x_offset=x_offset)
                for line in lines:
                    y_position = check_page_break(y_position)
                    c.drawString(x_offset, y_position, line)
                    y_position -= 15

                lines = wrap_text(f"Alleles: {mutation['Allele']}", width, 10, x_offset=x_offset)
                for line in lines:
                    y_position = check_page_break(y_position)
                    c.drawString(x_offset, y_position, line)
                    y_position -= 15

                y_position -= 20
        else:
            c.drawString(x_offset, y_position, "No mutation information available.")
            y_position -= 20

        y_position -= 40
        draw_full_line(c, y_position, width=500, x_offset=x_offset, line_padding=20)
        y_position -= 40

    c.showPage()
    c.save()
    pdf_content = buffer.getvalue()
    buffer.close()
    return pdf_content

In [82]:
def extract_valid_genes_from_document(file_path):
    extracted_genes = extract_genes_from_document(file_path)
    valid_genes = []

    for gene in extracted_genes:
        gene_info = get_gene_info_ensembl(gene)
        if gene_info:
            valid_genes.append(gene)

    return valid_genes

In [83]:
def get_consequences_from_user():
    so_terms = [
        "transcript_ablation",
        "splice_acceptor_variant",
        "splice_donor_variant",
        "stop_gained",
        "frameshift_variant",
        "stop_lost",
        "start_lost",
        "transcript_amplification",
        "feature_elongation",
        "feature_truncation",
        "inframe_insertion",
        "inframe_deletion",
        "missense_variant",
        "protein_altering_variant",
        "splice_donor_5th_base_variant",
        "splice_region_variant",
        "splice_donor_region_variant",
        "splice_polypyrimidine_tract_variant",
        "incomplete_terminal_codon_variant",
        "start_retained_variant",
        "stop_retained_variant",
        "synonymous_variant",
        "coding_sequence_variant",
        "mature_miRNA_variant",
        "5_prime_UTR_variant",
        "3_prime_UTR_variant",
        "non_coding_transcript_exon_variant",
        "intron_variant",
        "NMD_transcript_variant",
        "non_coding_transcript_variant",
        "coding_transcript_variant",
        "upstream_gene_variant",
        "downstream_gene_variant",
        "TFBS_ablation",
        "TFBS_amplification",
        "TF_binding_site_variant",
        "regulatory_region_ablation",
        "regulatory_region_amplification",
        "regulatory_region_variant",
        "intergenic_variant"
    ]

    print("\nValid Mutation Consequences:")
    for term in so_terms:
        print(f"- {term}")

    while True:
        consequences_input = input("Enter the mutation consequences you are interested in (comma-separated): ").split(',')

        consequences_input = [term.strip() for term in consequences_input]

        if all(consequence in so_terms for consequence in consequences_input):
            return consequences_input
        else:
            print("Invalid input. Please enter only the valid consequences from the list above.")

In [84]:
def genetic_counseling_assistant():
    while True:
        choice = input("Do you want to upload a file with gene names? (y/n): ").lower()

        while choice not in ["y", "n"]:
            print("Invalid choice. Please enter 'y' for yes or 'n' for no.")
            choice = input("Do you want to upload a file with gene names? (y/n): ").lower()

        if choice == "y":
            while True:
                file_path = get_file_path_colab()
                if file_path.endswith(('.pdf', '.docx', '.txt')):
                    genes = extract_valid_genes_from_document(file_path)
                    break
                else:
                    print("Unsupported file type. Please upload a PDF, DOCX, or TXT file.")

        elif choice == "n":
            gene_name = input("Enter a gene name: ")
            genes = [gene_name]

        genes = list(set(genes))

        while True:
            try:
                mutation_limit = int(input("Enter the number of mutations to retrieve (default 5): ") or 5)
                break
            except ValueError:
                print("Invalid input. Please enter a valid number.")

        consequences = get_consequences_from_user()

        genes_data = []

        for gene in genes:
            gene_info = get_gene_info_ensembl(gene)
            gene_function = get_gene_function(gene)
            mutations = get_filtered_mutation_data_ensembl(gene, mutation_limit, consequences)

            genes_data.append((gene_info, gene_function, mutations))

        if genes_data:
            report_content = generate_report(genes_data)
            with open("genetic_counseling_report.pdf", "wb") as f:
                f.write(report_content)
            print("Genetic Counseling Report has been generated and saved as 'genetic_counseling_report.pdf'")

        complete_context = ""
        for gene_data in genes_data:
            gene_info, gene_function, mutations = gene_data
            complete_context += f"Gene Information: {gene_info if gene_info else 'None'}\n"
            complete_context += f"Gene Function: {gene_function if gene_function else 'None'}\n"
            complete_context += f"Mutation Data: {mutations if mutations else 'None'}\n\n"

        while True:
            follow_up_question = input("Do you have any follow-up questions related to genetic counseling? (yes/no): ").lower()

            while follow_up_question not in ['yes', 'no']:
                print("Invalid input. Please enter 'yes' or 'no'.")
                follow_up_question = input("Do you have any follow-up questions related to genetic counseling? (yes/no): ").lower()

            if follow_up_question == "yes":
                question = input("Please enter your follow-up question: ")

                response = chatbot_with_groq(question, complete_context)
                print(f"Chatbot Response: {response}")

            elif follow_up_question == "no":
                print("Thank you for using the Genetic Counseling Assistant! Have a great day!")
                break

        continue_session = input("Would you like to process another set of gene data? (yes/no): ").lower()

        while continue_session not in ['yes', 'no']:
            print("Invalid input. Please enter 'yes' or 'no'.")
            continue_session = input("Would you like to process another set of gene data? (yes/no): ").lower()

        if continue_session != "yes":
            print("Goodbye!")
            break

In [ ]:
genetic_counseling_assistant()